#Introduction

## Assortativity

Assortativity is described as the tendency of vertices to connect or ‘attach’ to vertices with similar properties in a graph. This is effectively a measure of homophily in a network. 

***High Assortativity*** </br>
Advantage:  

1.   more robust to destructive events like the loss of vertices </br>

Disadvantage:

1.   because of their concentration they can be inefficient in terms of information flow
2.   in organizational settings they can be problematic for diverse interaction and experience



The assortativity coefficient of a graph is a measure of the extent to which vertices with the same properties connect to each other. 

The assortativity coefficient of a graph ranges between 
−
1
 and 
1
, just like a correlation coefficient. Assortativity coefficients close to 
1
 indicate that there is very high likelihood of two vertices with the same property being connected. This is called an assortative network. Assortativity coefficients close to 
−
1
 indicate that there is very low likelihood of two vertices with the same property being connected. This is called a disassortative network. Networks with assortativity coefficients close to zero are neither assortative nor disassortative and are usually described as having neutral assortativity.



In [6]:
import networkx as nx
import pandas as pd

# load data
# get data
schoolfriends_edgelist = pd.read_csv(
  "https://ona-book.org/data/schoolfriends_edgelist.csv"
)
schoolfriends_vertices = pd.read_csv(
  "https://ona-book.org/data/schoolfriends_vertices.csv"
)

# create undirected facebook graph
schoolfriends_fb = nx.from_pandas_edgelist(
  df = schoolfriends_edgelist[
    schoolfriends_edgelist.type == 'facebook'
  ],
  source = "from",
  target = "to"
)

# create directed reported graph
schoolfriends_rp = nx.from_pandas_edgelist(
  df = schoolfriends_edgelist[
    schoolfriends_edgelist.type == 'reported'
  ],
  source = "from",
  target = "to",
  create_using=nx.DiGraph()
)

# add class vertex attribute to both graphs
class_attr = dict(zip(schoolfriends_vertices['id'], 
schoolfriends_vertices['class']))
nx.set_node_attributes(schoolfriends_fb, name = "class", 
values = class_attr)
nx.set_node_attributes(schoolfriends_rp, name = "class", 
values = class_attr)

### Categorical or nominal assortativity

This suggests moderate assortativity, or moderate likelihood that students in the same class will be Facebook friends. Now let’s compare to ‘real’ reported friendships.



In [2]:
nx.attribute_assortativity_coefficient(schoolfriends_fb, "class")

0.3833666875368265

In [7]:
nx.attribute_assortativity_coefficient(schoolfriends_rp, "class")

0.7188918572576617

We see substantially higher assortativity by class for reported friendships, indicating that being in the same class is more strongly associated with developing a reported school friendship.

### Degree Assortativity

The most common form of numerical assortativity is degree assortativity. A high degree assortativity is a measure of preferential attachment in organizations, where highly connected vertices are connected with each other and a large number of vertices with low degree make up the remainder of the network.



In [8]:
nx.degree_assortativity_coefficient(schoolfriends_fb)

0.02462443563586026

In [9]:
nx.degree_assortativity_coefficient(schoolfriends_rp)

0.30981226480406526

We see that real-life friendships are moderately assortative in this data, whereas Facebook friendships are approximately neutral. This indicates that more popular students have a stronger tendency in real-life to be friends with other popular students.



Although it is a relatively recent concept, assortativity is a useful measure in understanding people or organizational networks. Highly assortative networks demonstrate resilience in that knowledge, community and other social capital are concentrated in a strong core, and disruptions such as departures of actors from those networks are less likely to affect the network as a whole. However, such networks also demonstrate characteristics which are counterproductive to diversity and inclusion and can represent challenging environments for new entrants to adjust to.



## Similarity

In certain networks where information on vertex properties is not available, it may make sense to infer that two vertices are similar in some way if their immediate networks are very similar.

##Vertex similarity

Often we are not lucky enough to have really rich ground truth information on the vertices in a network. This is particularly the case if we need to analyze people networks under anonymization constraints.

In some networks, we can conclude that vertices are similar in some way if they share very similar immediate connections. For example, in our workfrance graph, if two vertices have first degree networks that overlap to a large degree, it would not be unreasonable to infer a likelihood that those two vertices are from the same department.



***vertex similarity coefficient***


1.   ***Jaccard similarity coefficient***: the number of vertices who are neighbors of both 
v1
 and 
v2
 divided by the number of vertices who are neighbors of at least one of 
v1
 and 
v2.

2.   ***dice similarity coefficient***: twice the number of vertices who are neighbors of both 
v
1
 and 
v
2
 divided by the sum of the degree centralities of 
v
1
 and 
v
2
. Thus, common neighbors are double counted in this method.

3. ***inverse log-weighted similarity coefficient***: the sum of the inverse logs of the degrees of the common neighbors of 
v
1
 and 
v
2
. This definition asserts that common neighbors that have high degree in the network are ‘less valuable’ in detecting similarity because there is a higher likelihood that they would be a common neighbor simply by chance.




In this example, we calculate the Jaccard coefficients for two pairs of vertices in schoolfriends_fb.



In [10]:
jaccards = nx.jaccard_coefficient(G = schoolfriends_fb, 
ebunch = [(883, 132), (63, 991)])
sorted(jaccards)

[(63, 991, 0.15384615384615385), (883, 132, 0.30612244897959184)]

Dice similarity and inverse log weighted similarity coefficients can be calculated by creating functions

In [13]:
# function for dice similarity
def dice_coefficient(G, ebunch = None):
      def dicesim(u, v):
        total_degree = nx.degree(G, u) + nx.degree(G, v)
        if total_degree == 0:
            return 0
        return 2*len(list(nx.common_neighbors(G, u, v))) / total_degree
      
      if ebunch is None:
        ebunch = nx.non_edges(G)
      return ((u, v, dicesim(u, v)) for u, v in ebunch)

    
  
# test
dice = dice_coefficient(G = schoolfriends_fb, 
ebunch = [(883, 132), (63, 991)])
sorted(dice)

[(63, 991, 0.26666666666666666), (883, 132, 0.46875)]

In [14]:
import math 

# function for inverse log weighted similarity
def invlogweight_coefficient(G, ebunch = None):
      def invlogwsim(u, v):
        logw = [1/math.log(nx.degree(G, w)) 
                for w in nx.common_neighbors(G, u, v)]
        if logw == 0:
            return 0
        return sum(i for i in logw)
      
      if ebunch is None:
        ebunch = nx.non_edges(G)
      return ((u, v, invlogwsim(u, v)) for u, v in ebunch)

    
  
# test
invlogw = invlogweight_coefficient(G = schoolfriends_fb, 
ebunch = [(883, 132), (63, 991)])
sorted(invlogw)

[(63, 991, 0.5728002621049868), (883, 132, 4.566433503199232)]

##Graph Similarity

The general question of whether two graphs which contain vertices representing different entities still exhibit similar internal patterns and structures is of great interest in computer science and image recognition. 

The graph isomorphism problem, for example, is a computational problem concerned with finding a function that exactly maps the vertices of one graph onto other.

In organizational network analysis we are usually more concerned with comparing graphs where the vertices represent the same entities (usually people), and where the vertex set is identical but where the edge set may be different. 

Let 
G
1
=
(
V
,
E
1
)
 and 
G
2
=
(
V
,
E
2
)
 be two graphs with the same vertex set. The Jaccard similarity of 
G
1
 and 
G
2
 is the number of edges in both 
E
1
 and 
E
2
 divided by the number of edges in at least one of 
E
1
 and 
E
2
.

Note that a Jaccard similarity of 1 means that both graphs have identical edge sets and so are identical in structure, and a similarity of 0 means that both graphs have no edges in common. 

Finally, to calculate Jaccard similarity of two edge sets, a simple function can be written:



In [15]:
# create function for Jaccard edge set similarity
def jaccard_edgeset_similarity(G1, G2):
  setG1 = set(G1.edges)
  intersection = len(setG1.intersection(G2.edges))
  union = len(setG1.union(G2.edges))
  
  if union == 0:
    return 0
  return intersection/union

# recreate schoolfriends_fb as directed graph to compare
schoolfriends_fb_dir = schoolfriends_fb.to_directed()

# test
jaccard_edgeset_similarity(schoolfriends_fb_dir, schoolfriends_rp)

0.09727385377942999

We see that there is only about 10% similarity between the Facebook friendships and the ‘real’ reported friendships.

